**Connect google drive**

In [ ]:
!nvidia-smi

Tue Apr 12 01:57:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.02 MiB | 18.41 MiB/s, done.
Resolving deltas: 100% (10356/10356), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "swimming_pool" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-04-12 01:58:53--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  56.7MB/s    in 2.7s    

2022-04-12 01:58:56 (56.7 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/images/1.PNG   
  inflating: data/obj/images/1.txt   
  inflating: data/obj/images/10.PNG  
  inflating: data/obj/images/10.txt  
  inflating: data/obj/images/100.PNG  
  inflating: data/obj/images/100.txt  
  inflating: data/obj/images/101.PNG  
  inflating: data/obj/images/101.txt  
  inflating: data/obj/images/102.PNG  
  inflating: data/obj/images/102.txt  
  inflating: data/obj/images/107.PNG  
  inflating: data/obj/images/107.txt  
  inflating: data/obj/images/108.PNG  
  inflating: data/obj/images/108.txt  
  inflating: data/obj/images/109.PNG  
  inflating: data/obj/images/109.txt  
  inflating: data/obj/images/11.PNG  
  inflating: data/obj/images/11.txt  
  inflating: data/obj/images/113.PNG  
  inflating: data/obj/images/113.txt  
  inflating: data/obj/images/114.PNG  
  inflating: data/obj/images/114.txt  
  inflating: data/obj/images/115.PNG  
  inflating: data/obj/images/115.txt  
  inflating: data/obj/images/117.

In [ ]:
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

In [ ]:
import glob
images_list = glob.glob("data/obj/images/*.PNG")
print(images_list)

['data/obj/images/158.PNG', 'data/obj/images/93.PNG', 'data/obj/images/91.PNG', 'data/obj/images/96.PNG', 'data/obj/images/163.PNG', 'data/obj/images/43.PNG', 'data/obj/images/29.PNG', 'data/obj/images/192.PNG', 'data/obj/images/148.PNG', 'data/obj/images/27.PNG', 'data/obj/images/119.PNG', 'data/obj/images/4.PNG', 'data/obj/images/25.PNG', 'data/obj/images/1.PNG', 'data/obj/images/127.PNG', 'data/obj/images/94.PNG', 'data/obj/images/151.PNG', 'data/obj/images/137.PNG', 'data/obj/images/41.PNG', 'data/obj/images/159.PNG', 'data/obj/images/30.PNG', 'data/obj/images/114.PNG', 'data/obj/images/33.PNG', 'data/obj/images/162.PNG', 'data/obj/images/76.PNG', 'data/obj/images/108.PNG', 'data/obj/images/51.PNG', 'data/obj/images/167.PNG', 'data/obj/images/154.PNG', 'data/obj/images/39.PNG', 'data/obj/images/24.PNG', 'data/obj/images/59.PNG', 'data/obj/images/40.PNG', 'data/obj/images/73.PNG', 'data/obj/images/22.PNG', 'data/obj/images/123.PNG', 'data/obj/images/120.PNG', 'data/obj/images/175.PN

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 91654, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.063959, iou_loss = 0.000000, total_loss = 0.063959 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.672358), count: 6, class_loss = 0.808422, iou_loss = 0.676655, total_loss = 1.485076 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.006331, iou_loss = 0.000000, total_loss = 0.006331 
 total_bbox = 91660, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.802758), count: 1, class_loss = 0.036347, iou_loss = 0.034564, total_loss = 0.070911 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.662635), count: 2, class_loss = 0.485176, iou_loss = 0.158001, total_loss = 0.643178 
v3 (mse 